# 特徴量を追加していく用のCSVファイルを吐き出すための処理

### 2016年1月1日から2017年3月14日まで（439日）
### 439日×821店舗=360,419 行　visitorsがNanになっている部分もあります。

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
air_visit_data = pd.read_csv('../data/air_visit_data.csv',parse_dates=['visit_date'])

In [33]:
sample_submission=pd.read_csv('../data/sample_submission.csv')

In [34]:
sample_submission['air_store_id']=sample_submission['id'].str[:-11]

In [35]:
temp=sample_submission.groupby('air_store_id').count()
temp.reset_index(inplace=True)

In [37]:
air_visit_data_critical=air_visit_data[air_visit_data['air_store_id'].isin(temp['air_store_id'])]

In [38]:
air_visit_data_critical.shape

(250468, 3)

In [98]:
train_data_temp=air_visit_data_critical[(air_visit_data_critical['visit_date']>='2016-01-01')&(air_visit_data_critical['visit_date']<='2017-03-14')]

In [99]:
train_data_temp.describe(include='all')

,air_store_id,visit_date,visitors
count,222488,222488,222488.000000
unique,821,439,NaN
top,air_5c817ef28f236bdf,2017-03-03 00:00:00,NaN
freq,438,786,NaN
first,NaN,2016-01-01 00:00:00,NaN
last,NaN,2017-03-14 00:00:00,NaN
mean,NaN,NaN,20.806048
std,NaN,NaN,16.513870
min,NaN,NaN,1.000000
25%,NaN,NaN,9.000000


In [123]:
data_info=pd.read_csv('../data/date_info.csv', parse_dates=['calendar_date'])

In [124]:
temp=data_info[(data_info['calendar_date']>='2016-01-01')&(data_info['calendar_date']<='2017-03-14')]
temp.rename(columns={'calendar_date':'visit_date'}, inplace=True)

C:\Users\keisuke_ida\Miniconda3\envs\recruit_py36\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [125]:
temp2=train_data_temp.groupby(['air_store_id']).count()
temp2.reset_index(inplace=True)

In [126]:
temp['key']=1
temp2['key']=1
temp3=pd.merge(temp, temp2, on='key')

C:\Users\keisuke_ida\Miniconda3\envs\recruit_py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
temp4=temp3.ix[:, ('visit_date_x', 'air_store_id')].sort_values(['air_store_id', 'visit_date_x'])
temp4=temp4.ix[:, [1,0]]
temp4.rename(columns={'visit_date_x':'visit_date'}, inplace=True)

C:\Users\keisuke_ida\Miniconda3\envs\recruit_py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\keisuke_ida\Miniconda3\envs\recruit_py36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [128]:
features_format = pd.merge(temp4, train_data_temp, on=['air_store_id', 'visit_date'], how='left')

In [130]:
features_format.reset_index(drop=True).to_csv('../data/features_format.csv', index=False)